In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
import numpy as np
from rdkit.Chem import Draw
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import sklearn.metrics as metrics
import scikitplot as skplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import precision_recall_curve

In [ ]:
data = pd.read_csv('/home/valeriia/bakalarka/filvaleriia/LXRb_ch25_curated_DW_2.csv')
# copy raw data
data_new = data.copy()
data_new["Molecule"] = [Chem.MolFromSmiles(mol) for mol in data["smiles"]]


#colmun['Molecule'] invert to binary systems
data_new['bin'] = [np.array(AllChem.GetMorganFingerprintAsBitVect(i,2, nBits=1024)) for i in data_new['Molecule']]
data_new.info()



In [ ]:
#classifikation model thre = 6 and i used aktive/inaktive column
#this model predict active/inactive compounds
random_state = 20


#add new columns, number of columns
data_new['number'] = data_new['chembl_id'].rank(method='min')
lab_enc = preprocessing.LabelEncoder()
data_new['number']= lab_enc.fit_transform(data_new['number'])

#divided date for two sets: train and test
train, test = train_test_split(data_new, test_size = 0.25, random_state = random_state)
train, test = train.copy(), test.copy()
x_train = np.asarray([x for x in train['bin']])
x_test = np.asarray([x for x in test['bin']]) 
y_train = [y for y in train['category']]
y_test = [y for y in test['category']]
lab_enc = preprocessing.LabelEncoder()
training_Y = lab_enc.fit_transform(y_train)
testing_Y = lab_enc.fit_transform(y_test)


#RandomForesrClassifier
crf = RandomForestClassifier(n_estimators=101, max_depth=4, random_state=random_state)
crf.fit(x_train,training_Y)

# prediction on test set
crf_predict = crf.predict(x_test)

#CLassification:
   #accuracy, spe, sen, MCC,
    #confusion matrices (true positive, true negative, false positive, false negative (TP, TN, FP, FN))
    
#confusion_matrix
print("Confusion_matrix:")
conf_matrix = confusion_matrix(crf.predict(x_test), testing_Y)
print(conf_matrix)
TN, FP, FN, TP = conf_matrix.ravel()
d = {' ': ['actual+', 'actual-'], 'predicted+':[TP, FN], 'predicted-':[FP, TN]}
df = pd.DataFrame(data=d)
print("\n")
print(df)
print("\n")

#Accuracy
print("Accuracy:",metrics.accuracy_score(testing_Y, crf.predict(x_test)))
print("\n")
print("=== Classification Report ===")
print(classification_report(testing_Y, crf_predict))

# Performance metrics
errors = abs(crf_predict - testing_Y)
print('Average absolute error:', round(np.mean(errors), 2), 'degrees.')
# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
print("Sensitivity:", round(TPR,3))
# Specificity or true negative rate
TNR = TN/(TN+FP) 
print("Specifity:", round(TNR, 3))
#MCC
MCC = ((TP*TN)-(FP*FN))/((TP+FP)*(FN+TN)*(FP+TN)*(TP+FN))**(1/2)
print("MMC:", round(MCC, 3))
# Precision or positive predictive value
PPV = TP/(TP+FP)
print("Positive predictive", round(PPV,3))
# Negative predictive value
NPV = TN/(TN+FN)
print("Negative predictive", round(NPV,3))
# Fall out or false positive rate
FPR = FP/(FP+TN)
print("False positive", round(FPR,3))
# False negative rate
FNR = FN/(TP+FN)
print("False negative", round(FNR,3))
# False discovery rate
FDR = FP/(TP+FP)
print("False discovery", round(FDR,3))
# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print("Overall accuracy", round(ACC,3))




y_score1 = crf.predict_proba(x_test)[:,1]
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(testing_Y, y_score1)
print('roc_auc_score for RandomForestClassification: ', round(roc_auc_score(testing_Y, y_score1),4))

#PLOT ROC curves
plt.subplots(1, figsize=(10,10))
plt.title('ROC cuves')
plt.plot(false_positive_rate1, true_positive_rate1)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate(TP)')
plt.xlabel('False Positive Rate(FP)')
plt.show()
